Since this course is on programming and not on statistics *per se*, we won't go into the details of statistical tests (there are many courses on that online, including free MOOCs on [coursera](https://www.coursera.org/) and [udacity](https://www.udacity.com/)). Rather, we will look at how these tests can be implemented, how you can use the power of simulations to get a better understanding of the test's limitations, and at bootstrapping statistics (in the next lesson).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import scipy.stats as stats

Pure statistical tests are not Python's strongest side, it is historically the field where *R* is dominating, so if you do primarily statistics, especially something compicated, it might pay off to learn a bit of *R* (you can even call it from within Python using `rpy2` module). However, most of the things you can do in Python without much effort. 

In Python main statistical packages are `scipy.stats` ([documentation](https://docs.scipy.org/doc/scipy/reference/stats.html) is of average quality, but it does have a quite comprehensive [tutorial](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html)) and build on top of it `statsmodels` ([documentation](http://www.statsmodels.org/stable/index.html) is also average, but has a [tutorial](http://www.statsmodels.org/stable/index.html#table-of-contents) and contains many examples [here](http://www.statsmodels.org/stable/examples/index.html) and [here](https://github.com/statsmodels/statsmodels/wiki/Examples)). While `scipy.stats` implements much of the basic functionality, like different tests, distribution functions, etc, `statsmodels` provides a higher-level application programming interface (API) and a nice output with comprehensive statistical summaries. It also provides an interface to write models in *R*-like manner (it uses [`patsy`](http://patsy.readthedocs.io/en/latest/) library) and works well with `pandas` `DataFrames`. `scipy.stats` frequently requires messy input.

But the first thing we will do is a little demonstration of basic principles of statistics, using a simulation. We will generate two samples of normally distributed values, with mean of 0 and standard deviation of 1. Then we will run a Student's t-test on them:

In [ ]:
x = np.random.normal(0, 1, size=20)
y = np.random.normal(0, 1, size=20)
result = stats.ttest_ind(x,y)
result.pvalue

And we got some p-value. Let's do the same test 10,000 times and save all the p-values in an array:

In [ ]:
# number of iterations
n_iterations = 10000

# preallocate array which will hold the p-values
p_values = pd.Series(np.full(shape=n_iterations, fill_value=np.nan))

for i in range(n_iterations):
    x = np.random.normal(0, 1, size=20)
    y = np.random.normal(0, 1, size=20)
    p_values[i] = stats.ttest_ind(x,y).pvalue

Now we will plot a histogram of the p-value distribution. But before we do it, think about what you would expect to see. Remember, that the two distributions had no difference.

In [ ]:
plt.hist(p_values,20)
plt.xlabel('p-value');

As you can see, the distribution of p-values is uniform under null hypothesis. Now, let's say we pick a significance threshold (alpha) of 0.05. What is the proportion of p-values that falls below alpha? Think about what you'd expect here.

In [ ]:
# compute the proportion of p values that fall below the 0.05 level
power = np.mean(p_values<0.05)
print(power)

When your distribution of p-values is uniform, it follows that the proportion of your p-values that fall below a given alpha threshold is exactly alpha. This is your false positive rate under null hypothesis.

Now we will do the same, but this time, we will have an effect -- one of the distributions will be centered at 0.5:

In [ ]:
# number of iterations
n_iterations = 10000

# preallocate array which will hold the p-values
p_values = pd.Series(np.full(shape=n_iterations, fill_value=np.nan))

for i in range(n_iterations):
    x = np.random.normal(0.5, 1, size=20)
    y = np.random.normal(0, 1, size=20)
    p_values[i] = stats.ttest_ind(x,y).pvalue

Again, we plot the histogram of p-values. Think about what you expect to change now that we have a real effect in our test.

In [ ]:
plt.hist(p_values,50)
plt.xlabel('p-value');

Now let's compute the power. Since all of our hypotheses have an effect, the power is the proportion of p-values that fall below alpha:

In [ ]:
# compute the power of this test at the 0.05 level
power = np.mean(p_values<0.05)
print(power)

Basically, this simulation tells us that with this sample size of 20, at the alpha level of 0.05 we will detect an effect of `0.5` in ~30-35% of samples. Using math, you can calculate it precisely, but simulations are much more straightforward.

It is important to understand that these concepts lie at the heart of statistics, independent of the test or model you are using. That is why it is crucial that you undestand the interplay between the 4 parameters that we mentioned in this section: **alpha**, **beta** (and therefore power, because power = 1-beta), **N** and **effect size**. They are bound together in a sense than if you define 3 of them, the forth will be fixed.

For example, let's say I have collected a certain (normally distributed) sample of size 20 for each of the two groups. And I want to know what is the minimum effect I can detect with 80% power at alpha level of 0.05 (or 0.01 or 0.001). To get a better understanding of that, I will run a simulation for different effects and see the power curves, as we call them. To do it, I will basically run the same thing we did before, but instead of using just one effect size, I will use multiple (ranging from 0 to 1.9, with a step of 0.1). I will calculate power for each of the effects at the alpha levels of 0.05, 0.01 and 0.001. (Beware that if you have a slow computer, it might take a while. You may want to skip this and just look at the screen.)

In [ ]:
# number of iterations
n_iterations = 10000

# sample size
n = 20

# effect sizes
effects = np.arange(0,2,0.1)

# powers will be saved in these Series (specific for each alpha level)
power_005=pd.Series(np.zeros_like(effects), dtype=float)
power_001=pd.Series(np.zeros_like(effects), dtype=float)
power_0001=pd.Series(np.zeros_like(effects), dtype=float)

for j, effect in enumerate(effects):
    
    p_values = pd.Series(np.full(shape=n_iterations, fill_value=np.nan))
    for i in range(10000):
        x=np.random.normal(effect, 1, size = n)
        y=np.random.normal(0, 1, size = n)
        p_values[i] = stats.ttest_ind(x,y).pvalue
    
    power_005[j]=np.mean(p_values<0.05)
    power_001[j]=np.mean(p_values<0.01)
    power_0001[j]=np.mean(p_values<0.001)
    
    print(j,end=', ')

In [ ]:
plt.plot(effects,power_005, label='0.05')
plt.plot(effects,power_001, label='0.01')
plt.plot(effects,power_0001, label='0.001')
plt.xlabel('Effect size (Cohen\'s d)')
plt.ylabel('Power')
plt.grid('on')
plt.title('N=' + str(n))
plt.legend()

Remember that there are actually 4 parameters, and you can fix any 2 of them and look at the relationship of the other 2. Here we fixed sample size (N) and alpha levels (we did simulation for 3 different alpha levels). But we can also fix effect size and alpha, and get something very useful: dependence of power on the sample size. Let's say you want your effect to be at least 1 standard deviation, which would translate to Cohen's d = 1. Let's simulate the same thing, but for different sample sizes:

In [ ]:
# number of iterations
n_iterations = 10000

# sample size
sample_sizes = np.arange(4,50,4)

# effect sizes
effect = 1

# powers will be saved in these Series (specific for each alpha level)
power_005=pd.Series(np.zeros_like(sample_sizes), dtype=float)
power_001=pd.Series(np.zeros_like(sample_sizes), dtype=float)
power_0001=pd.Series(np.zeros_like(sample_sizes), dtype=float)

for j, n in enumerate(sample_sizes):
    
    p_values = pd.Series(np.full(shape=n_iterations, fill_value=np.nan))
    for i in range(10000):
        x=np.random.normal(effect, 1, size = n)
        y=np.random.normal(0, 1, size = n)
        p_values[i] = stats.ttest_ind(x,y).pvalue
    
    power_005[j]=np.mean(p_values<0.05)
    power_001[j]=np.mean(p_values<0.01)
    power_0001[j]=np.mean(p_values<0.001)
    
    print(j,end=', ')

In [ ]:
plt.plot(sample_sizes,power_005, label='0.05')
plt.plot(sample_sizes,power_001, label='0.01')
plt.plot(sample_sizes,power_0001, label='0.001')
plt.xlabel('Sample size, n')
plt.ylabel('Power')
plt.grid('on')
plt.title('Cohen\'s d=' + str(effect))
plt.legend()

Let's see an example with real data. Let's say we have this dataset:

> **Dataset description**: Twenty-two patients undergoing cardiac bypass surgery were randomized to one of three ventilation groups:

> (1) Patients received 50% nitrous oxide and 50% oxygen mixture continuously for 24 h.

> (2) Patients received a 50% nitrous oxide and 50% oxygen mixture only during the operation.

> (3) Patients received no nitrous oxide but received 35-50% oxygen for 24 h.

>The data show red cell folate levels for the three groups after 24h ventilation.

In [ ]:
data = pd.read_csv('data/cardiac_bypass.csv')
data

If this is your final dataset, you'd probably want to to an ANOVA test to see if one of the groups is different.

This dataset is in tidy format, however `scipy.stats.f_oneway` which implements 1-way ANOVA, requires all groups to be passed as separate arguments (as you can see in the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html)), which is a messy input. One way to do this is to separate our groups, like so:

In [ ]:
groups = []
for _, group_data in data.groupby('group')['level']:
    groups.append(group_data.values)
    
# a list of values separated by group
groups

>**Note**: we used `_` in the loop definition because when you iterate through a groupby object, it will first return the *name* of the group, and only then the values; we assign the values to the variable `group_data`, but we don't use the name of the group in our loop, so I save it to the *_* variable, to demonstrate to myself and those who are reading the code that I am not using it.

>**Pro-tip**: This can be rewritten an a list comprehension:

>        groups = [group_data.values for _, group_data in data.groupby('group')['level']]

But I need to submit to the function not a list of arrays, but each array separately as an argument. I can to it like that:

In [ ]:
F, p = stats.f_oneway(groups[0], groups[1], groups[2])
print('The p-value of one way ANOVA: ' + str(p))

But there is a shorter way, I can use an *unpacking* operator `*` on the list, which is used inside a function call and submits each element of the list (or a tuple) as a separate argument:

In [ ]:
F, p = stats.f_oneway(*groups)
print('The p-value of one way ANOVA: ' + str(p))

Ok, so at this point you might be happy or you might be sad, but this is what you have. Usually after an ANOVA test, you need to do a post-hoc test on groups pair-wise to see which group is different. Since in this case you have a small sample and a border-line p-value from the ANOVA test, instead of Student's t-test you might want to use something more conservative. In particular, you might want to use Welch's t-test (which doesn't require equal variance assumption) or Welch's t-test on ranks (which doesn't require also normality).

> Welch's t-test is implemented in `scipy.stats.ttest_ind()` with `equal_var` parameter set to `False`. Test on ranks is not implemented with a function, but we can easily convert our data to ranks with `Series.rank()` method.

In [ ]:
# Welch t-test
stats.ttest_ind(groups[0], groups[1], equal_var=False)

In [ ]:
# convert data to ranks 
data['level_rank'] = data['level'].rank()
data.head()

In [ ]:
# split ranked level by group
groups_rank = [group_data.values for _, group_data in data.groupby('group')['level_rank']]

# Welch t-test on ranks
stats.ttest_ind(groups_rank[0], groups_rank[1], equal_var=False)

As you see, a more conservative Welch t-test on ranks doesn't give significance. If this is your primary dataset, you're quite stuck at this point, at least with regards to frequentist statistics. However, if this is you pilot study, you might want to explore your dataset further, for example, you might want to look at what sample size would reliably detect the effect you have in the pilot study.

In [ ]:
means = data.groupby('group')['level'].mean()
stds = data.groupby('group')['level'].std()
n_samples = data.groupby('group')['level'].aggregate(len)

In [ ]:
sample_sizes = np.arange(4,32,2)
power_005 = np.zeros_like(sample_sizes).astype(float)
power_001 = np.zeros_like(sample_sizes).astype(float)
power_0005 = np.zeros_like(sample_sizes).astype(float)

for k, n_sim in enumerate(sample_sizes):

    n_iterations = 1000
    p_vals = np.full(n_iterations, np.nan)
    for i in range(n_iterations):
        samples = []
        for j in range(3):
            samples.append(np.random.normal(means[j],stds[j],size=n_sim))
            _, p_vals[i] = stats.f_oneway(*samples)

    power_005[k] = (p_vals<0.05).mean()
    power_001[k] = (p_vals<0.01).mean()
    power_0005[k] = (p_vals<0.005).mean()
    print(k, end=', ')

In [ ]:
plt.plot(sample_sizes,power_005,label='0.05')
plt.plot(sample_sizes,power_001,label='0.01')
plt.plot(sample_sizes,power_0005,label='0.005')
plt.xlabel('sample size')
plt.ylabel('power')
plt.grid('on')
plt.legend()